core5_02_failure_metric_recompute.ipynb

목적:
- Core 2~3에서 사용했던 불안정 지표들을
  원본 trace로부터 다시 계산
- 이전 결과와 '일치 가능한 값'이 재현되는지 확인

중요:
- 여기서 계산한 지표는 '새로운 지표'가 아님
- 실패가 우연이 아니라 구조적으로 재현됨을 보이기 위함

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

TRACE_PATH = Path("../artifact/core2/prediction_trace.csv")
RISK_TRACE_PATH = Path("../artifact/core3/core3_instability_risk_with_trace.csv")

print("prediction_trace.csv exists:", TRACE_PATH.exists())
print("instability_risk_with_trace.csv exists:", RISK_TRACE_PATH.exists())

trace_df = pd.read_csv(TRACE_PATH)

print("=== prediction_trace shape ===")
print(trace_df.shape)

print("\n=== columns ===")
print(trace_df.columns.tolist())

trace_df.head()

prediction_trace.csv exists: True
instability_risk_with_trace.csv exists: True
=== prediction_trace shape ===
(60, 8)

=== columns ===
['antibody_key', 'step', 'sequence_current', 'pred_score', 'pred_score_delta', 'decision', 'mutation_id_applied', 'intervention_count_cum']


,antibody_key,step,sequence_current,pred_score,pred_score_delta,decision,mutation_id_applied,intervention_count_cum
0,GDPa1-001,1,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,2.013943,0.013943,MUTATE,GDPa1-001_mut0_1,1
1,GDPa1-001,2,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.966444,-0.047499,HOLD,NaN,1
2,GDPa1-001,3,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.943947,-0.022497,HOLD,NaN,1
3,GDPa1-001,4,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.916268,-0.027679,HOLD,NaN,1
4,GDPa1-001,5,EAKIIFEVDWQCADHITYAVHVQIRWKAGQMKFHMEDPENNYKCRV...,1.939915,0.023647,MUTATE,GDPa1-001_mut1_5,2


In [2]:
# decision 변화 여부로 toggle_event 재계산
trace_df = trace_df.sort_values(["antibody_key", "step"]).reset_index(drop=True)

trace_df["prev_decision"] = (
    trace_df.groupby("antibody_key")["decision"].shift(1)
)

trace_df["toggle_event_recomputed"] = (
    trace_df["decision"] != trace_df["prev_decision"]
).astype(int)

trace_df[[
    "antibody_key",
    "step",
    "decision",
    "prev_decision",
    "toggle_event_recomputed"
]].head(10)

toggle_summary = (
    trace_df
    .groupby("antibody_key")["toggle_event_recomputed"]
    .agg(
        toggle_count="sum",
        total_steps="count"
    )
    .reset_index()
)

toggle_summary["toggle_rate"] = (
    toggle_summary["toggle_count"] / toggle_summary["total_steps"]
)

toggle_summary # toggle_rate (항체별 / 전체) 재계산

overall_toggle_rate = trace_df["toggle_event_recomputed"].mean()
overall_toggle_rate

0.4166666666666667

In [3]:
def compute_burst_lengths(df):
    bursts = []
    current_burst = 0
    
    for decision in df["decision"]:
        if decision == "MUTATE":
            current_burst += 1
        else:
            if current_burst > 0:
                bursts.append(current_burst)
                current_burst = 0
    if current_burst > 0:
        bursts.append(current_burst)
    return bursts # burst_length 재계산 (연속 MUTATE 구간)

burst_records = []

for ab, g in trace_df.groupby("antibody_key"):
    bursts = compute_burst_lengths(g)
    for b in bursts:
        burst_records.append({
            "antibody_key": ab,
            "burst_length": b
        })

burst_df = pd.DataFrame(burst_records)

burst_df.head()

,antibody_key,burst_length
0,GDPa1-001,1
1,GDPa1-001,3
2,GDPa1-001,1
3,GDPa1-001,2
4,GDPa1-001,2


In [4]:
burst_summary = (
    burst_df
    .groupby("antibody_key")["burst_length"]
    .agg(
        mean_burst="mean",
        max_burst="max",
        burst_count="count"
    )
    .reset_index()
)

burst_summary # burst 통계 요약

,antibody_key,mean_burst,max_burst,burst_count
0,GDPa1-001,1.8,3,5
1,GDPa1-045,3.5,6,4
2,GDPa1-183,1.5,3,4


In [5]:
intervention_summary = (
    trace_df
    .groupby("antibody_key")["intervention_count_cum"]
    .agg(
        final_intervention_count="max",
        mean_intervention_count="mean"
    )
    .reset_index()
)

intervention_summary # intervention_count 분포 재계산

,antibody_key,final_intervention_count,mean_intervention_count
0,GDPa1-001,9,4.50
1,GDPa1-045,14,6.35
2,GDPa1-183,6,2.65


In [6]:
if RISK_TRACE_PATH.exists():
    risk_df = pd.read_csv(RISK_TRACE_PATH)
    
    print("=== instability_risk trace loaded ===")
    print(risk_df.shape)
    
    risk_stats = {
        "mean_risk": risk_df["instability_risk"].mean(),
        "std_risk": risk_df["instability_risk"].std(),
        "p90_risk": np.percentile(risk_df["instability_risk"], 90),
        "p95_risk": np.percentile(risk_df["instability_risk"], 95),
    }
    
    risk_stats
else:
    risk_df = None
    risk_stats = None
    print("No instability_risk trace found.") # instability_risk 분포 확인

=== instability_risk trace loaded ===
(60, 9)


In [7]:
if risk_df is not None and "pred_score" in risk_df.columns:
    corr = np.corrcoef(
        risk_df["pred_score"],
        risk_df["instability_risk"]
    )[0, 1]
    corr
else:
    corr = None # pred_score와 instability_risk 상관

In [8]:
failure_metrics = (
    toggle_summary
    .merge(burst_summary, on="antibody_key", how="left")
    .merge(intervention_summary, on="antibody_key", how="left")
)

failure_metrics # 항체별 실패 지표 통합 테이블 생성

overall_row = {
    "antibody_key": "ALL",
    "toggle_count": trace_df["toggle_event_recomputed"].sum(),
    "total_steps": len(trace_df),
    "toggle_rate": overall_toggle_rate,
    "mean_burst": burst_df["burst_length"].mean() if not burst_df.empty else np.nan,
    "max_burst": burst_df["burst_length"].max() if not burst_df.empty else np.nan,
    "burst_count": len(burst_df),
    "final_intervention_count": trace_df["intervention_count_cum"].max(),
    "mean_intervention_count": trace_df["intervention_count_cum"].mean()
}

failure_metrics = pd.concat(
    [failure_metrics, pd.DataFrame([overall_row])],
    ignore_index=True
)

failure_metrics

OUTPUT_DIR = Path("../artifact/core5")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

output_path = OUTPUT_DIR / "recomputed_failure_metrics.csv"
failure_metrics.to_csv(output_path, index=False)

output_path

PosixPath('../artifact/core5/recomputed_failure_metrics.csv')

In [9]:
print("""
[Core 5 Failure Metric Recompute Complete]

- toggle_rate, burst_length, intervention_count는
  원본 trace로부터 다시 계산되었다.
- 이전 Core에서 관찰된 불안정 지표는
  동일한 구조에서 재현된다.
- 실패는 우연이 아니라 계산으로 다시 만들어진다.
""")


[Core 5 Failure Metric Recompute Complete]

- toggle_rate, burst_length, intervention_count는
  원본 trace로부터 다시 계산되었다.
- 이전 Core에서 관찰된 불안정 지표는
  동일한 구조에서 재현된다.
- 실패는 우연이 아니라 계산으로 다시 만들어진다.

